In [6]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer
from nltk.metrics import edit_distance
from nltk.stem.porter import *
stemmer = PorterStemmer()
#from nltk.stem.snowball import SnowballStemmer #0.003 improvement but takes twice as long as PorterStemmer
#stemmer = SnowballStemmer('english')
import re
#import enchant
import random

In [19]:
print "data preparation"
num_train=74067
df_train = pd.read_csv("../input/x_train.csv", encoding="ISO-8859-1")#, index_col=1)
df_test=pd.read_csv("../input/x_test.csv", encoding="ISO-8859-1")#,index_col=1)
coswithst=pd.read_csv("../data/coswithst_m3.csv", encoding="ISO-8859-1",header=None)
coswithst.columns = ['distance']
cws_trn=coswithst.iloc[:num_train]
cws_tst=coswithst.iloc[num_train:]



data preparation


In [20]:
df_train

,product_uid,id,matchCount,search_term_count,product_title_count,stHitRatio,titleHitRatio,matchCount_desc,product_description_count,stHitRatio_desc,descHitRatio,brandMatch,product_title,search_term,product_description,brand,category
0,100001,2,1,2,6,0.500000,0.166667,1,94,0.500000,0.010638,0,simpson strong tie 12 gaug angl,angl bracket,angl make joint stronger also provid consist s...,simpson strong tie,3
1,100001,3,1,2,6,0.500000,0.166667,1,94,0.500000,0.010638,0,simpson strong tie 12 gaug angl,l bracket,angl make joint stronger also provid consist s...,simpson strong tie,3
2,100002,9,1,1,12,1.000000,0.083333,1,121,1.000000,0.008264,0,behr premium textur deckov 1 gal sc 141 tugboa...,deck,behr premium textur deckov inchesov solid colo...,behr premium textur deckov,3
3,100005,16,1,3,11,0.333333,0.090909,1,72,0.333333,0.013889,0,delta vero 1 handl shower faucet trim kit inch...,rain shower head,updat bathroom delta vero singl handl shower f...,delta,2
4,100005,17,2,2,11,1.000000,0.181818,2,72,1.000000,0.027778,0,delta vero 1 handl shower faucet trim kit inch...,shower faucet,updat bathroom delta vero singl handl shower f...,delta,2
5,100006,18,1,2,12,0.500000,0.083333,2,324,1.000000,0.006173,0,whirlpool 1 9 cu ft rang convect microwav inch...,convect otr,achiev delici result almost effortless whirlpo...,whirlpool,1
6,100006,20,1,2,12,0.500000,0.083333,1,324,0.500000,0.003086,0,whirlpool 1 9 cu ft rang convect microwav inch...,microwav stove,achiev delici result almost effortless whirlpo...,whirlpool,1
7,100006,21,1,1,12,1.000000,0.083333,1,324,1.000000,0.003086,0,whirlpool 1 9 cu ft rang convect microwav inch...,microwav,achiev delici result almost effortless whirlpo...,whirlpool,1
8,100007,23,2,2,10,1.000000,0.200000,2,97,1.000000,0.020619,0,lithonia light quantum 2 light black led emerg...,emerg light,quantum adjust 2 light led black emerg light u...,lithonia light,2
9,100009,27,2,2,13,1.000000,0.153846,2,56,1.000000,0.035714,0,hous fara 3/4 inch x 3 inch x 8 ft mdf flute case,mdf 3/4,get hous fara 3/4 inch x 3 inch x 8 ft mdf flu...,hous fara,4


In [21]:
id_test = df_test['id']
id_train = df_train['id']
y_train = pd.read_csv("../input/y_train.csv", encoding="ISO-8859-1",header=None)

x_train =df_train[:]
x_test = df_test[:]
x_train= pd.concat([x_train, cws_trn], axis=1)
x_test = pd.concat([x_test.reset_index(drop=True),cws_tst.reset_index(drop=True)], axis=1)

print "data preparation done!"

data preparation done!


In [24]:
list(x_train.columns.values)

[u'product_uid',
 u'id',
 u'matchCount',
 u'search_term_count',
 u'product_title_count',
 u'stHitRatio',
 u'titleHitRatio',
 u'matchCount_desc',
 u'product_description_count',
 u'stHitRatio_desc',
 u'descHitRatio',
 u'brandMatch',
 u'product_title',
 u'search_term',
 u'product_description',
 u'brand',
 u'category',
 'distance']

In [50]:
x_train1=x_train[['matchCount',
 'search_term_count',
 'product_title_count',
 'stHitRatio',
 'titleHitRatio',
 'matchCount_desc',
 'product_description_count',
 'stHitRatio_desc',
 'descHitRatio',
 'brandMatch','distance']].copy()

In [51]:
x_test1=x_test[['matchCount',
 'search_term_count',
 'product_title_count',
 'stHitRatio',
 'titleHitRatio',
 'matchCount_desc',
 'product_description_count',
 'stHitRatio_desc',
 'descHitRatio',
 'brandMatch','distance']].copy()

In [52]:
x_train1

,matchCount,search_term_count,product_title_count,stHitRatio,titleHitRatio,matchCount_desc,product_description_count,stHitRatio_desc,descHitRatio,brandMatch,distance
0,1,2,6,0.500000,0.166667,1,94,0.500000,0.010638,0,0.048668
1,1,2,6,0.500000,0.166667,1,94,0.500000,0.010638,0,-0.079769
2,1,1,12,1.000000,0.083333,1,121,1.000000,0.008264,0,0.131103
3,1,3,11,0.333333,0.090909,1,72,0.333333,0.013889,0,0.030917
4,2,2,11,1.000000,0.181818,2,72,1.000000,0.027778,0,0.191109
5,1,2,12,0.500000,0.083333,2,324,1.000000,0.006173,0,0.111782
6,1,2,12,0.500000,0.083333,1,324,0.500000,0.003086,0,0.312819
7,1,1,12,1.000000,0.083333,1,324,1.000000,0.003086,0,0.118773
8,2,2,10,1.000000,0.200000,2,97,1.000000,0.020619,0,0.027768
9,2,2,13,1.000000,0.153846,2,56,1.000000,0.035714,0,0.028609


In [53]:
def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['id','relevance','search_term','product_title','product_description','product_info','attr','brand']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [54]:
class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [67]:
from sklearn.metrics import mean_squared_error, make_scorer
import re
import pandas as pd
import random
from sklearn import ensemble
import sklearn.grid_search as gs
from sklearn.cross_validation import StratifiedKFold
random.seed(2016)

def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)
import xgboost as xgb
gbm = xgb.XGBClassifier()
gbm_params = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': range(1,12),
    'max_depth': [1,2,5,10,15,20]
}

gridxb = gs.GridSearchCV(gbm, gbm_params,scoring=RMSE,cv=4,verbose=20,n_jobs=-1)

In [66]:
gridxb.fit(x_train1, y_train)

Fitting 4 folds for each of 132 candidates, totalling 528 fits


PicklingError: Can't pickle <function fmean_squared_error at 0x134ed39b0>: it's not the same object as __main__.fmean_squared_error

In [57]:
print("Best parameters found by grid search:")
print(gridxb.best_params_)
print("Best CV score:")
print(gridxb.best_score_)
print(gridxb.best_score_ + 0.47003199274)

Best parameters found by grid search:
{'n_estimators': 8, 'learning_rate': 0.01, 'max_depth': 1}
Best CV score:
-0.541629633298
-0.0715976405577
[CV] n_estimators=1, learning_rate=0.1, max_depth=1 ..................

[CV] n_estimators=1, learning_rate=0.1, max_depth=1 ..................

[CV] n_estimators=9, learning_rate=0.01, max_depth=20 ................
[CV] n_estimators=9, learning_rate=0.01, max_depth=20 ................
[CV] n_estimators=9, learning_rate=0.01, max_depth=20 ................
[CV]  n_estimators=7, learning_rate=0.01, max_depth=20, score=-0.608245 -  19.1s[CV]  n_estimators=1, learning_rate=0.1, max_depth=1, score=-0.541313 -   0.5s[CV] n_estimators=1, learning_rate=0.1, max_depth=1 ..................
[CV]  n_estimators=1, learning_rate=0.1, max_depth=1, score=-0.544268 -   0.5s[CV] n_estimators=1, learning_rate=0.1, max_depth=1 ..................
[CV]  n_estimators=9, learning_rate=0.01, max_depth=20, score=-0.624434 -  24.6s[CV]  n_estimators=9, learning_rate=0.01

In [58]:
#grid_search_forest.fit(x_train, y_train)
y_pred = gridxb.predict(x_test1)
pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission5_wc.csv',index=False)

[CV]  n_estimators=4, learning_rate=0.1, max_depth=10, score=-0.638287 -   5.5s[CV] n_estimators=4, learning_rate=0.1, max_depth=10 .................
[CV]  n_estimators=10, learning_rate=0.1, max_depth=5, score=-0.607279 -   7.0s[CV] n_estimators=10, learning_rate=0.1, max_depth=5 .................
[CV]  n_estimators=4, learning_rate=0.1, max_depth=15, score=-0.585106 -   8.3s[CV] n_estimators=7, learning_rate=0.1, max_depth=15 .................
[CV]  n_estimators=4, learning_rate=0.1, max_depth=15, score=-0.592075 -   8.4s

[CV]  n_estimators=4, learning_rate=0.1, max_depth=10, score=-0.577885 -   5.5s
[CV]  n_estimators=10, learning_rate=0.1, max_depth=5, score=-0.567545 -   7.0s
[CV]  n_estimators=7, learning_rate=0.1, max_depth=15, score=-0.603435 -  14.6s
[CV] n_estimators=7, learning_rate=0.1, max_depth=15 .................
[CV] n_estimators=6, learning_rate=0.1, max_depth=10 .................

[CV] n_estimators=1, learning_rate=0.1, max_depth=10 .................

[CV] n_estimat

In [59]:
# test RMSE: 0.54535
# pervious test rmse without category: 0.54313
# test error without category: 0.54535